In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False
DataList = "PetImages/Traing_Data_List.npy"

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}
    trainingData = []
    catCount = 0
    dogCount = 0
    
    def make_training_data(self):
        for label in self.LABELS:    # label -> path
            print(label)
            for f in tqdm(os.listdir(label)):    # f -> img name, ex: 1.jpg 2.jpg ...
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    # one hot, np.eye(2)[0] -> [1, 0], np.eye(5)[2] -> [0, 0, 1, 0, 0]
                    self.trainingData.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catCount += 1
                    elif label == self.DOGS:
                        self.dogCount += 1
                except Exception as e:
                    pass
                    #print("IMG ERROR: ", e)
        
        np.random.shuffle(self.trainingData)
        np.save(DataList, self.trainingData)
        print("Cats: ", self.catCount)
        print("Dogs: ", self.dogCount)
            

if REBUILD_DATA:
    dogsvscats = DogsVSCats()
    dogsvscats.make_training_data()
                
training_data = np.load(DataList, allow_pickle=True)
print(f"Data load down! {len(training_data)} imgs")

Data load down! 24946 imgs


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128*2*2, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


In [3]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0 
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]
print("DATA SPLIT DONE")

DATA SPLIT DONE


In [4]:
if torch.cuda.is_available():
    print(torch.cuda.device_count())
    device = torch.device("cuda:0")
    print("RUNNING GPU")
else:
    device = torch.device("cpu")
    print("RUNNING CPU")

1
RUNNING GPU


In [5]:
net = Net().to(device)

In [8]:
def train(net):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    
    BATCH_SIZE = 100
    EPOCHS = 10
    
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1,1, 50, 50).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)

            net.zero_grad()
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss}")

train(net)

  2%|█▊                                                                                | 5/225 [00:00<00:05, 42.83it/s]

Epoch: 0, Loss: 0.1451861709356308


  2%|█▊                                                                                | 5/225 [00:00<00:05, 40.43it/s]

Epoch: 1, Loss: 0.11028838902711868


  2%|█▊                                                                                | 5/225 [00:00<00:05, 42.84it/s]

...

Epoch: 7, Loss: 0.044056132435798645


  2%|█▊                                                                                | 5/225 [00:00<00:05, 42.70it/s]

Epoch: 8, Loss: 0.049394167959690094


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:06<00:00, 35.41it/s]

Epoch: 9, Loss: 0.032756201922893524


In [9]:
def test(net): 
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i].view(-1, 1, 50, 50).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print(f"Acc -> {round(correct/total, 3)}%")
    
test(net)

100%|█████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:02<00:00, 853.12it/s]

Acc -> 0.737%
